In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googlesearch import search
from urllib.parse import urlparse
import time
import logging
import re
import json
import random

# Deshabilita las advertencias de certificados no verificados
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Lista de User-Agents para rotar
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
]

def scraping_data(query, query_type):
    """
    Función unificada para realizar web scraping, ya sea por URL o por nombre.
    """
    # Selecciona un User-Agent al azar para cada solicitud
    headers = {'User-Agent': random.choice(user_agents)}
    url_final = None

    if query_type == "url":
        url_final = query
    elif query_type == "nombre":
        try:
            resultados_busqueda = list(search(query, stop=1, pause=random.uniform(2, 4)))
            if not resultados_busqueda:
                logging.warning(f"No se encontró URL para el nombre: {query}")
                return None
            url_final = resultados_busqueda[0]
        except Exception as e:
            logging.error(f"Error en la búsqueda de Google para '{query}': {e}")
            return None
    else:
        logging.error("Tipo de consulta no válido. Use 'url' o 'nombre'.")
        return None

    logging.info(f"URL final a procesar: {url_final}")

    # Obtener el dominio de la URL
    dominio = urlparse(url_final).netloc

    # Realizar web scraping con manejo de errores y reintentos
    response = None
    for _ in range(3):
        try:
            response = requests.get(url_final, headers=headers, timeout=30, verify=False)
            response.raise_for_status()
            break
        except requests.exceptions.RequestException as e:
            logging.error(f"Intento fallido para {url_final}: {e}. Reintentando...")
            time.sleep(random.uniform(5, 10))

    if not response:
        logging.error(f"No se pudo acceder a la URL después de varios intentos: {url_final}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # --- Extracción de información general ---
    titulo = soup.title.text.strip() if soup.title else None
    descripcion = soup.find('meta', {'name': 'description'})
    descripcion = descripcion['content'].strip() if descripcion else None
    palabras_clave = soup.find('meta', {'name': 'keywords'})
    palabras_clave = palabras_clave['content'].strip() if palabras_clave else None
    encabezados = [h.text.strip() for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
    enlaces = [link.get('href') for link in soup.find_all('a', href=True)]
    parrafos = [p.text.strip() for p in soup.find_all('p')]
    imagenes = [img.get('src') for img in soup.find_all('img', src=True)]
    tablas = []
    for table in soup.find_all('table'):
        table_data = []
        for row in table.find_all('tr'):
            row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
            table_data.append(row_data)
        tablas.append(table_data)

    # --- Nuevos campos de búsqueda ---
    razon_social = None
    tax_id = None
    brand = None
    email = None
    telefono = None
    direccion = None
    productos_servicios = []

    # Búsqueda en datos estructurados de Schema.org
    for script in soup.find_all('script', type='application/ld+json'):
        if script.string:
            try:
                data = json.loads(script.string)
                if isinstance(data, list):
                    for item in data:
                        if item.get('@type') in ['Organization', 'LocalBusiness']:
                            razon_social = item.get('name')
                            tax_id = item.get('taxID')
                            brand = item.get('brand')
                            if item.get('address') and isinstance(item['address'], dict):
                                direccion = item['address'].get('streetAddress')
                            if item.get('contactPoint') and isinstance(item['contactPoint'], list):
                                for cp in item['contactPoint']:
                                    if cp.get('contactType') == 'customer service' or cp.get('contactType') == 'general':
                                        telefono = cp.get('telephone')
                                        email = cp.get('email')
                        elif item.get('@type') in ['Product', 'Service']:
                            productos_servicios.append(item.get('name'))
                elif data.get('@type') in ['Organization', 'LocalBusiness']:
                    razon_social = data.get('name')
                    tax_id = data.get('taxID')
                    brand = data.get('brand')
                    if data.get('address') and isinstance(data['address'], dict):
                        direccion = data['address'].get('streetAddress')
                    if data.get('contactPoint') and isinstance(data['contactPoint'], list):
                        for cp in data['contactPoint']:
                            if cp.get('contactType') == 'customer service' or cp.get('contactType') == 'general':
                                telefono = cp.get('telephone')
                                email = cp.get('email')
                elif data.get('@type') in ['Product', 'Service']:
                    productos_servicios.append(data.get('name'))
            except json.JSONDecodeError:
                continue

    # Si no se encuentra razón social en Schema.org, buscar en el h1
    if not razon_social:
        razon_social_element = soup.find('h1')
        razon_social = razon_social_element.text.strip() if razon_social_element else None

    # Si no se encuentra email o teléfono, buscar por patrones
    if not email:
        email_match = re.search(r'[\w\.-]+@[\w\.-]+\.[\w\.]+', str(soup))
        email = email_match.group(0) if email_match else None

    if not telefono:
        telefono_match = re.search(r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', str(soup))
        telefono = telefono_match.group(0) if telefono_match else None

    return {
        'nombre_buscado': query if query_type == 'nombre' else None,
        'url_buscada': query if query_type == 'url' else None,
        'url_encontrada': url_final,
        'dominio': dominio,
        'razon_social': razon_social,
        'taxID': tax_id,
        'brand': brand,
        'email': email,
        'telefono': telefono,
        'direccion': direccion,
        'productos_servicios': productos_servicios,
        'titulo': titulo,
        'descripcion': descripcion,
        'palabras_clave': palabras_clave,
        'encabezados': encabezados,
        'enlaces': enlaces,
        'parrafos': parrafos,
        'imagenes': imagenes,
        'tablas': tablas
    }

# --- Nuevo listado de empresas con URLs y/o nombres ---
listado_empresas = [
    {"nombre": "ENERCA S.A. E.S.P."}, # Solo nombre
    {"url": "https://www.anla.gov.co/"}, # Solo URL
    {"nombre": "JURISDICCIÓN ESPECIAL PARA LA PAZ (JEP)", "url": "https://www.jep.gov.co/JEP/Paginas/Jurisdiccion-Especial-para-la-Paz.aspx"}, # Ambos
    {"url": "https://www.esap.edu.co/?ref=elfrente.com.co"},
    {"nombre": "INSTITUTO NACIONAL DE VIGILANCIA DE MEDICAMENTOS Y ALIMENTOS (INVIMA)"},
    {"url": "https://www.fng.gov.co/"},
    {"nombre": "CORPORACIÓN AUTÓNOMA REGIONAL RIONEGRO-NARE (CORNARE)", "url": "https://www.cornare.gov.co/"},
    {"url": "https://www.tumaco-narino.gov.co/"},
    {"nombre": "LIGA COLOMBIANA CONTRA EL CÁNCER"},
    {"url": "https://www.inci.gov.co/"},
    {"nombre": "INSTITUTO COLOMBIANO AGROPECUARIO (ICA)"},
    {"url": "https://www.balboa-risaralda.gov.co/"},
    {"nombre": "GOBERNACIÓN DEL VALLE DEL CAUCA"}, # Solo nombre
    {"url": "http://lamerced-caldas.gov.co/"},
    {"nombre": "AGENCIA PARA LA REINCORPORACIÓN Y LA NORMALIZACIÓN", "url": "https://www.reincorporacion.gov.co/es"},
    {"url": "http://esecentrodesaludzetaquira.gov.co/"},
    {"nombre": "INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI (IGAC)"},
    {"nombre": "SENADO DE LA REPÚBLICA DE COLOMBIA"},
    {"url": "https://www.volcan.com.pe/en/"},
    {"url": "https://ecuaquimica.com.ec/"},
    {"url": "https://www.adelca.com/"},
    {"url": "https://gocorp.com/"},
    {"url": "https://www.sancarlos.com.ec/"},
    {"url": "https://www.facebook.com/eljuridivisionlicores/"},
    {"url": "https://www.cajaarequipa.pe/"},
    {"url": "https://www.ajegroup.com/"},
    {"url": "https://www.exalmar.com.pe/en/"},
    {"url": "https://www.tstt.co.tt/"},
    {"url": "https://www.bancoagricola.com/"},
    {"url": "https://www.cafebritt.com/?srsltid=AfmBOopz--QU1VspyVC7tUUg5S_v1aDH86rJpME5PzNZxRR8AR17sxm6"},
    {"url": "https://www.banrural.com.gt/site/conocenos/grupofinancierobanrural"},
    {"url": "https://my.clevelandclinic.org/health/diseases/17032-glioblastoma"},
    {"url": "https://eng.disagro.com/"},
    {"url": "https://www.bancocuscatlan.com/persona"},
    {"url": "https://www.bam.org/"},
    {"url": "https://agrokasa.com/en/"},
    {"url": "https://www.copaair.com/"},
    {"url": "https://bhd.com.do/"},
    {"url": "https://motorcredito.com.do/"},
    {"url": "https://refidomsa.com/"},
    {"url": "https://cibao.com.do/banca-personal"},
    {"url": "https://www.bcie.org/"},
    {"url": "https://www.tigo.com.hn/"},
    {"url": "https://www.millicom.com/"},
    {"url": "https://www.banesco.com.pa/"},
    {"url": "https://www.libertypr.com/en/"},
    {"url": "https://www.digitel.com.ve/la-corporacion"}
]

# Realizar el scraping para cada entrada del listado
resultados_empresas = []
for item in listado_empresas:
    if "url" in item and item["url"]:
        # Si tiene URL, la procesa directamente
        resultado = scraping_data(item["url"], "url")
    elif "nombre" in item and item["nombre"]:
        # Si solo tiene nombre, lo busca en Google
        resultado = scraping_data(item["nombre"], "nombre")
    else:
        logging.warning("Entrada de lista vacía, saltando...")
        continue

    if resultado:
        resultados_empresas.append(resultado)

    # Retardo aleatorio para evitar bloqueos
    time.sleep(random.uniform(5, 10))

# Guardar los resultados en un DataFrame de pandas y un archivo Excel
df_resultados = pd.DataFrame(resultados_empresas)
df_resultados.to_excel("/content/resultados_y_o_mejorado.xlsx", index=False)

print("Resultados guardados en /content/resultados_y_o_mejorado.xlsx")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googlesearch import search
from urllib.parse import urlparse
import time
import logging
import re
import json
import random

# Deshabilita las advertencias de certificados no verificados
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Lista de User-Agents para rotar
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
]

def scraping_data(query, query_type):
    """
    Función unificada para realizar web scraping, ya sea por URL o por nombre.
    """
    # Selecciona un User-Agent al azar para cada solicitud
    headers = {'User-Agent': random.choice(user_agents)}
    url_final = None

    if query_type == "url":
        url_final = query
    elif query_type == "nombre":
        try:
            resultados_busqueda = list(search(query, stop=1, pause=random.uniform(2, 4)))
            if not resultados_busqueda:
                logging.warning(f"No se encontró URL para el nombre: {query}")
                return None
            url_final = resultados_busqueda[0]
        except Exception as e:
            logging.error(f"Error en la búsqueda de Google para '{query}': {e}")
            return None
    else:
        logging.error("Tipo de consulta no válido. Use 'url' o 'nombre'.")
        return None

    logging.info(f"URL final a procesar: {url_final}")

    dominio = urlparse(url_final).netloc

    response = None
    for _ in range(3):
        try:
            response = requests.get(url_final, headers=headers, timeout=30, verify=False)
            response.raise_for_status()
            break
        except requests.exceptions.RequestException as e:
            logging.error(f"Intento fallido para {url_final}: {e}. Reintentando...")
            time.sleep(random.uniform(5, 10))

    if not response:
        logging.error(f"No se pudo acceder a la URL después de varios intentos: {url_final}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    titulo = soup.title.text.strip() if soup.title else None
    descripcion = soup.find('meta', {'name': 'description'})
    descripcion = descripcion['content'].strip() if descripcion else None
    palabras_clave = soup.find('meta', {'name': 'keywords'})
    palabras_clave = palabras_clave['content'].strip() if palabras_clave else None
    encabezados = [h.text.strip() for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
    enlaces = [link.get('href') for link in soup.find_all('a', href=True)]
    parrafos = [p.text.strip() for p in soup.find_all('p')]
    imagenes = [img.get('src') for img in soup.find_all('img', src=True)]
    tablas = []
    for table in soup.find_all('table'):
        table_data = []
        for row in table.find_all('tr'):
            row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
            table_data.append(row_data)
        tablas.append(table_data)

    razon_social = None
    tax_id = None
    brand = None
    email = None
    telefono = None
    direccion = None
    productos_servicios = []

    for script in soup.find_all('script', type='application/ld+json'):
        if script.string:
            try:
                data = json.loads(script.string)
                if isinstance(data, list):
                    for item in data:
                        if item.get('@type') in ['Organization', 'LocalBusiness']:
                            razon_social = item.get('name')
                            tax_id = item.get('taxID')
                            brand = item.get('brand')
                            if item.get('address') and isinstance(item['address'], dict):
                                direccion = item['address'].get('streetAddress')
                            if item.get('contactPoint') and isinstance(item['contactPoint'], list):
                                for cp in item['contactPoint']:
                                    if cp.get('contactType') == 'customer service' or cp.get('contactType') == 'general':
                                        telefono = cp.get('telephone')
                                        email = cp.get('email')
                        elif item.get('@type') in ['Product', 'Service']:
                            productos_servicios.append(item.get('name'))
                elif data.get('@type') in ['Organization', 'LocalBusiness']:
                    razon_social = data.get('name')
                    tax_id = data.get('taxID')
                    brand = data.get('brand')
                    if data.get('address') and isinstance(data['address'], dict):
                        direccion = data['address'].get('streetAddress')
                    if data.get('contactPoint') and isinstance(data['contactPoint'], list):
                        for cp in data['contactPoint']:
                            if cp.get('contactType') == 'customer service' or cp.get('contactType') == 'general':
                                telefono = cp.get('telephone')
                                email = cp.get('email')
                elif data.get('@type') in ['Product', 'Service']:
                    productos_servicios.append(data.get('name'))
            except json.JSONDecodeError:
                continue

    if not razon_social:
        razon_social_element = soup.find('h1')
        razon_social = razon_social_element.text.strip() if razon_social_element else None

    if not email:
        email_match = re.search(r'[\w\.-]+@[\w\.-]+\.[\w\.]+', str(soup))
        email = email_match.group(0) if email_match else None

    if not telefono:
        telefono_match = re.search(r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', str(soup))
        telefono = telefono_match.group(0) if telefono_match else None

    return {
        'nombre_buscado': query if query_type == 'nombre' else None,
        'url_buscada': query if query_type == 'url' else None,
        'url_encontrada': url_final,
        'dominio': dominio,
        'razon_social': razon_social,
        'taxID': tax_id,
        'brand': brand,
        'email': email,
        'telefono': telefono,
        'direccion': direccion,
        'productos_servicios': productos_servicios,
        'titulo': titulo,
        'descripcion': descripcion,
        'palabras_clave': palabras_clave,
        'encabezados': encabezados,
        'enlaces': enlaces,
        'parrafos': parrafos,
        'imagenes': imagenes,
        'tablas': tablas
    }

# --- Nuevo listado de empresas con URLs y nombres ---
# Hemos estructurado la lista para que cada elemento sea un diccionario
# con una URL y un nombre, o solo uno de ellos.
listado_empresas = [
    {"url": "enerca.com.co", "nombre": "ENERCA S.A. E.S.P."},
    {"url": "anla.gov.co", "nombre": "AUTORIDAD NACIONAL DE LICENCIAS AMBIENTALES (ANLA)"},
    {"url": "jep.gov.co", "nombre": "JURISDICCIÓN ESPECIAL PARA LA PAZ (JEP)"},
    {"url": "esap.edu.co", "nombre": "ESCUELA SUPERIOR DE ADMINISTRACIÓN PÚBLICA (ESAP)"},
    {"url": "invima.gov.co", "nombre": "INSTITUTO NACIONAL DE VIGILANCIA DE MEDICAMENTOS Y ALIMENTOS (INVIMA)"},
    {"url": "fiduprevisora.com.co", "nombre": "FIDUPREVISORA S.A."},
    {"url": "cali.gov.co", "nombre": "ALCALDÍA DE SANTIAGO DE CALI"},
    {"url": "fng.gov.co", "nombre": "FONDO NACIONAL DE GARANTÍAS"},
    {"url": "cornare.gov.co", "nombre": "CORPORACIÓN AUTÓNOMA REGIONAL RIONEGRO-NARE (CORNARE)"},
    {"url": "tumaco-narino.gov.co", "nombre": "ALCALDÍA DE TUMACO, NARIÑO"},
    {"url": "cancer.gov.co", "nombre": "LIGA COLOMBIANA CONTRA EL CÁNCER"},
    {"url": "inci.gov.co", "nombre": "INSTITUTO NACIONAL PARA CIEGOS (INCI)"},
    {"url": "ica.gov.co", "nombre": "INSTITUTO COLOMBIANO AGROPECUARIO (ICA)"},
    {"url": "balboa-risaralda.gov.co", "nombre": "ALCALDÍA DE BALBOA, RISARALDA"},
    {"url": "valledelcauca.gov.co", "nombre": "GOBERNACIÓN DEL VALLE DEL CAUCA"},
    {"url": "fng.gov.co", "nombre": "FONDO NACIONAL DE GARANTÍAS"},
    {"url": "invima.gov.co", "nombre": "INSTITUTO NACIONAL DE VIGILANCIA DE MEDICAMENTOS Y ALIMENTOS (INVIMA)"},
    {"url": "lamerced-caldas.gov.co", "nombre": "ALCALDÍA DE LA MERCED, CALDAS"},
    {"url": "elretiro-antioquia.gov.co", "nombre": "ALCALDÍA DE EL RETIRO, ANTIOQUIA"},
    {"url": "reincorporacion.gov.co", "nombre": "AGENCIA PARA LA REINCORPORACIÓN Y LA NORMALIZACIÓN"},
    {"url": "mindefensa.gov.co", "nombre": "MINISTERIO DE DEFENSA NACIONAL"},
    {"url": "esecentrodesaludzetaquira.gov.co", "nombre": "ESE CENTRO DE SALUD DE ZETAQUIRA"},
    {"url": "igac.gov.co", "nombre": "INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI (IGAC)"},
    {"url": "fiduprevisora.com.co", "nombre": "FIDUPREVISORA S.A."},
    {"url": "fiduprevisora.com.co", "nombre": "FIDUPREVISORA S.A."},
    {"url": "igac.gov.co", "nombre": "INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI (IGAC)"},
    {"url": "senado.gov.co", "nombre": "SENADO DE LA REPÚBLICA DE COLOMBIA"},
    {"url": "pereira.gov.co", "nombre": "ALCALDÍA DE PEREIRA"},
    {"url": "igac.gov.co", "nombre": "INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI (IGAC)"},
    {"url": "fng.gov.co", "nombre": "FONDO NACIONAL DE GARANTÍAS"},
    {"url": "clinicainternacional.com.pe", "nombre": "Clínica Internacional"},
    {"url": "lima-airport.com", "nombre": "LIMA AIRPORT"},
    {"url": "antamina.com", "nombre": "Antamina"},
    {"url": "volcan.com.pe", "nombre": "Volcan Compañía Minera"},
    {"url": "bancoguayaquil.com", "nombre": "Banco Guayaquil"},
    {"url": "ecuaquimica.com", "nombre": "Ecuaquimica"},
    {"url": "adelca.com", "nombre": "Adelca"},
    {"url": "gerardoortiz.com", "nombre": "GO CORP"},
    {"url": "danec.com", "nombre": "Danec S.A."},
    {"url": "sancarlos.com.ec", "nombre": "Sociedad Agrícola e Industrial San Carlos S.A."},
    {"url": "cargill.com", "nombre": "Cargill"},
    {"url": "grupoeljuri.com", "nombre": "Eljuri Licores"},
    {"url": "cajaarequipa.pe", "nombre": "Caja Arequipa"},
    {"url": "ajegroup.com", "nombre": "Grupo AJE"},
    {"url": "latina.pe", "nombre": "Latina"},
    {"url": "exalmar.com.pe", "nombre": "Pesquera Exalmar SAA"},
    {"url": "edeeste.com.do", "nombre": "Edeeste"},
    {"url": "tstt.co.tt", "nombre": "Telecommunications Services of Trinidad & Tobago Limited (TSTT)"},
    {"url": "bancoagricola.com", "nombre": "Bancoagrícola"},
    {"url": "cafebritt.com", "nombre": "Café Britt"},
    {"url": "ccn.com.ni", "nombre": "Compañia Cervecera de Nicaragua S.A"},
    {"url": "laprensagrafica.com", "nombre": "La Prensa Grafica"},
    {"url": "banrural.com.gt", "nombre": "Grupo Financiero Banrural"},
    {"url": "gbm.net", "nombre": "GBM"},
    {"url": "bantrab.com.gt", "nombre": "Bantrab"},
    {"url": "disagro.com", "nombre": "DISAGRO"},
    {"url": "bancocuscatlan.com", "nombre": "Banco Cuscatlán El Salvador"},
    {"url": "bam.com.gt", "nombre": "Bam"},
    {"url": "agrokasa.com", "nombre": "Sociedad Agrícola Drokasa S.A."},
    {"url": "cbc.co", "nombre": "cbc"},
    {"url": "copaair.com", "nombre": "Copa Airlines"},
    {"url": "butterfieldgroup.com", "nombre": "Butterfield Group"},
    {"url": "tigo.com.co", "nombre": "Tigo Colombia"},
    {"url": "bhd.com.do", "nombre": "Banco BHD"},
    {"url": "motorcredito.com.do", "nombre": "Motor Crédito, S. A., Banco de Ahorro & Crédito"},
    {"url": "refidomsa.com.do", "nombre": "Refidomsa PDV, S.A."},
    {"url": "acap.com.do", "nombre": "Asociación Cibao de Ahorros y Préstamos"},
    {"url": "bcie.org", "nombre": "Banco Centroamericano de Integración Económica - BCIE"},
    {"url": "tigo.com.hn", "nombre": "Tigo Honduras."},
    {"url": "millicom.com", "nombre": "Millicom (Tigo)"},
    {"url": "bancocaribe.com.do", "nombre": "BANCO CARIBE"},
    {"url": "banesco.com.pa", "nombre": "Banesco Panamá"},
    {"url": "canalbank.com", "nombre": "Canalbank"},
    {"url": "libertypr.com", "nombre": "Liberty of Puerto Rico"},
    {"url": "digitel.com.ve", "nombre": "Corporación Digitel"}
]

# Realizar el scraping para cada entrada del listado
resultados_empresas = []
for item in listado_empresas:
    url_found = False

    # Intenta con la URL primero
    if "url" in item and item["url"]:
        # Se añade un protocolo si falta para evitar errores de requests
        full_url = item["url"]
        if not full_url.startswith('http://') and not full_url.startswith('https://'):
            full_url = 'https://' + full_url

        resultado = scraping_data(full_url, "url")
        if resultado and resultado['dominio']:
            # Si se obtiene un resultado válido de la URL, lo usa
            resultados_empresas.append(resultado)
            url_found = True
        else:
            logging.warning(f"La URL {full_url} falló. Intentando con el nombre.")

    # Si la URL falla o no está disponible, intenta con el nombre
    if not url_found and "nombre" in item and item["nombre"]:
        resultado = scraping_data(item["nombre"], "nombre")
        if resultado:
            resultados_empresas.append(resultado)

    # Retardo aleatorio para evitar bloqueos
    time.sleep(random.uniform(5, 10))

# Guardar los resultados en un DataFrame de pandas y un archivo Excel
df_resultados = pd.DataFrame(resultados_empresas)
df_resultados.to_excel("/content/resultados_completo.xlsx", index=False)

print("Resultados guardados en /content/resultados_completo.xlsx")